In [1]:
import pandas as pd
import numpy as np
import urllib
import os

from sklearn import model_selection

import tensorflow as tf
import keras as K

import matplotlib.pyplot as plt

from scipy.misc import toimage

%matplotlib inline

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
Xcept = K.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 1s 0us/step


In [6]:
Xcept.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

Включим в обучение последние 3 сверточных слоя из VGG16

In [0]:
Xcept.trainable = True

trainable = False

for layer in Xcept.layers:

    if layer.name == 'block5_conv1' and 'block5_conv2' and 'block5_conv3' :
        trainable = True

    layer.trainable = trainable

In [8]:
Xcept_fc = K.models.Sequential()

Xcept_fc.add(Xcept)

Xcept_fc.add(K.layers.Conv2D(256, (1,1), activation='relu'))
Xcept_fc.add(K.layers.Flatten())
Xcept_fc.add(K.layers.Dense(256, activation='relu'))
Xcept_fc.add(K.layers.Dense(128, activation='relu'))
Xcept_fc.add(K.layers.Dropout(0.2))
Xcept_fc.add(K.layers.Dense(1, activation='sigmoid'))

Xcept_fc.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
Xcept_fc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 256)         131328    
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               3211520   
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total para

In [14]:
batch_size = 32
target_size = (224, 224)

train_aug_gen = K.preprocessing.image.ImageDataGenerator(rescale=1/255, 
                                                               zoom_range=0.2,
                                                               channel_shift_range=0.2,
                                                               shear_range=0.2, 
                                                               brightness_range=(0.1, 0.9),
                                                               rotation_range=60,
                                                               width_shift_range=0.2, 
                                                               height_shift_range=0.2, 
                                                               fill_mode='nearest', )

val_aug_gen = K.preprocessing.image.ImageDataGenerator(rescale=1/255, 
                                                               zoom_range=0.1,
                                                               channel_shift_range=0.1,
                                                               shear_range=0.1, 
                                                               brightness_range=(0.3, 0.6),
                                                               rotation_range=60,
                                                               width_shift_range=0.1, 
                                                               height_shift_range=0.1, 
                                                               fill_mode='nearest', )

data_gen = K.preprocessing.image.ImageDataGenerator(rescale=1/255)

train_rand_gen = train_aug_gen.flow_from_directory('/content/gdrive/My Drive/NewBalance/Images/train',
                                                   target_size=target_size, class_mode='binary', batch_size=batch_size)

val_rand_gen = val_aug_gen.flow_from_directory('/content/gdrive/My Drive/NewBalance/Images/val', 
                                            target_size=target_size, class_mode='binary', batch_size=batch_size)

test_gen = data_gen.flow_from_directory('/content/gdrive/My Drive/NewBalance/Images/test',
                                        target_size=target_size, batch_size=batch_size, class_mode='binary')

Found 736 images belonging to 2 classes.
Found 184 images belonging to 2 classes.
Found 163 images belonging to 2 classes.


In [0]:
checkpoint_X = K.callbacks.ModelCheckpoint('/content/gdrive/My Drive/NewBalance/xcept.hdf5', monitor='val_acc', save_best_only=True, verbose=1 )

learning_rate_reduction_X = K.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5, min_lr=0, 
                                                          factor=0.5, mode='auto', verbose=1)

In [15]:
history_X = Xcept_fc.fit_generator(train_rand_gen, epochs=200, validation_data=val_rand_gen, 
                              use_multiprocessing=True,
                              steps_per_epoch=train_rand_gen.samples//batch_size, 
                              validation_steps=val_rand_gen.samples//batch_size,
                              callbacks=[checkpoint_X])

Epoch 1/200
23/23 [==============================] - 71s 3s/step - loss: 0.4238 - acc: 0.7921 - val_loss: 0.4232 - val_acc: 0.7625

Epoch 00001: val_acc did not improve from 0.76829
Epoch 2/200
23/23 [==============================] - 66s 3s/step - loss: 0.4367 - acc: 0.7853 - val_loss: 0.3801 - val_acc: 0.8224

Epoch 00002: val_acc improved from 0.76829 to 0.82237, saving model to /content/gdrive/My Drive/NewBalance/xcept.hdf5
Epoch 3/200
23/23 [==============================] - 62s 3s/step - loss: 0.3863 - acc: 0.8207 - val_loss: 0.4039 - val_acc: 0.8355

Epoch 00003: val_acc improved from 0.82237 to 0.83553, saving model to /content/gdrive/My Drive/NewBalance/xcept.hdf5
Epoch 4/200
23/23 [==============================] - 64s 3s/step - loss: 0.4226 - acc: 0.8071 - val_loss: 0.4521 - val_acc: 0.8026

Epoch 00004: val_acc did not improve from 0.83553
Epoch 5/200
23/23 [==============================] - 64s 3s/step - loss: 0.3878 - acc: 0.8234 - val_loss: 0.4840 - val_acc: 0.7961

Epoc

Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 626, in next_sample
    return six.next(_SHARED_SEQUENCES[uid])
  File "/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/iterator.py", line 100, in __next__
    return self.next(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/iterator.py", line 112, in next
    return self._get_batches_of_transformed_samples(index_ar

KeyboardInterrupt: ignored

In [17]:
Xcept_fc.evaluate_generator(test_gen, steps=test_gen.samples//batch_size)

[0.7155425685052653, 0.8931297714473637]

In [18]:
plt.plot(history_X.history['acc'], label='trian_acc')
plt.plot(history_X.history['val_acc'], label='val_acc')
plt.xlabel('epochs')
plt.ylabel('acc')
plt.legend()
plt.show()

NameError: ignored

In [0]:
plt.plot(history_X.history['loss'], label='trian_loss')
plt.plot(history_X.history['val_loss'], label='val_loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [0]:
Xcept_fc.load_weights('/content/gdrive/My Drive/NewBalance/xcept.hdf5')

In [25]:
Xcept_fc.evaluate_generator(test_gen, steps=test_gen.samples//batch_size)

[0.2920853516783651, 0.916030534351145]

In [0]:
model_json = Xcept_fc.to_json()
with open("/content/gdrive/My Drive/NewBalance/vgg16_tl.json", "w") as json_file:
  json_file.write(model_json)
Xcept.save_weights("/content/gdrive/My Drive/NewBalance/vgg16_tl.hdf5")